In [113]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [114]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [115]:
TRAIN_DIR = 'C:/VSCodeProjects/Face Emotion Recognition/images/train'
TEST_DIR = 'C:/VSCodeProjects/Face Emotion Recognition/images/test'

In [116]:
def create_dataFrame(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [117]:
train = pd.DataFrame()
train['image'], train['label'] = create_dataFrame(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [118]:
print(train)

                                                   image     label
0      C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
1      C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
2      C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
3      C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
4      C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
...                                                  ...       ...
28816  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
28817  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
28818  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
28819  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
28820  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise

[28821 rows x 2 columns]


In [119]:
test = pd.DataFrame()
test['image'], test['label'] = create_dataFrame(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [120]:
print(test)

                                                  image     label
0     C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
1     C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
2     C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
3     C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
4     C:/VSCodeProjects/Face Emotion Recognition/ima...     angry
...                                                 ...       ...
7061  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
7062  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
7063  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
7064  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise
7065  C:/VSCodeProjects/Face Emotion Recognition/ima...  surprise

[7066 rows x 2 columns]


In [121]:
from tqdm.notebook import tqdm

In [122]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)  # Convert list to a NumPy array before reshaping
    features = features.reshape(len(features),48,48,1) #hwhere depth = 1 since we're dealing with 2d images
    return features

In [123]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [124]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [125]:
x_train = train_features/255.0
x_test = test_features/255.0

In [126]:
from sklearn.preprocessing import LabelEncoder

In [127]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [128]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [129]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [130]:
model = Sequential()
# convulational layers
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation = 'softmax'))

In [131]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [132]:
model.fit(x = x_train, y = y_train, batch_size = 128, epochs = 50, validation_data = (x_test, y_test))

Epoch 1/50
226/226 [==============================] - 22s 95ms/step - loss: 1.8240 - accuracy: 0.2417 - val_loss: 1.8102 - val_accuracy: 0.2583
Epoch 2/50
226/226 [==============================] - 21s 95ms/step - loss: 1.7979 - accuracy: 0.2491 - val_loss: 1.7721 - val_accuracy: 0.2603
Epoch 3/50
226/226 [==============================] - 22s 96ms/step - loss: 1.7147 - accuracy: 0.3005 - val_loss: 1.6043 - val_accuracy: 0.3534
Epoch 4/50
226/226 [==============================] - 22s 99ms/step - loss: 1.5942 - accuracy: 0.3697 - val_loss: 1.4697 - val_accuracy: 0.4336
Epoch 5/50
226/226 [==============================] - 23s 102ms/step - loss: 1.4963 - accuracy: 0.4188 - val_loss: 1.3656 - val_accuracy: 0.4798
Epoch 6/50
226/226 [==============================] - 23s 103ms/step - loss: 1.4319 - accuracy: 0.4483 - val_loss: 1.3211 - val_accuracy: 0.4917
Epoch 7/50
226/226 [==============================] - 23s 103ms/step - loss: 1.3894 - accuracy: 0.4663 - val_loss: 1.2997 - val_accura

In [133]:
model_json = model.to_json()
with open("FacialEmotionDetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("FacialEmotionDetector.h5")

In [134]:
from keras.models import model_from_json

In [146]:
json_file = open("FacialEmotionDetector.json", 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("FacialEmotionDetector.h5")

In [147]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [148]:
def ef(image):
    img = load_img(image, color_mode = "grayscale")
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [154]:
image = 'C:/VSCodeProjects/Face Emotion Recognition/images/train/sad/3.jpg'
print("original image = sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction = ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

original image = sad
model prediction =  sad


NameError: name 'plt' is not defined

In [158]:
current_path = os.getcwd()
print(current_path)

C:\Users\Saksham
